In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

In [2]:
train_data3 = pd.read_csv('../data/curated/modelling/RS_Train_3.csv')
val_data3 = pd.read_csv('../data/curated/modelling/RS_Validate_3.csv')
test_data3 = pd.read_csv('../data/curated/modelling/RS_Test_3.csv')

train_data2 = pd.read_csv('../data/curated/modelling/RS_Train_2.csv')
val_data2 = pd.read_csv('../data/curated/modelling/RS_Validate_2.csv')
test_data2 = pd.read_csv('../data/curated/modelling/RS_Test_2.csv')

train_data1 = pd.read_csv('../data/curated/modelling/RS_Train_1.csv')
val_data1 = pd.read_csv('../data/curated/modelling/RS_Validate_1.csv')
test_data1 = pd.read_csv('../data/curated/modelling/RS_Test_1.csv')

In [3]:
train3_x = train_data3.drop(['3'], axis=1)
train3_y = train_data3['3']
val3_x = val_data3.drop(['3'], axis=1)
val3_y = val_data3['3']

train2_x = train_data2.drop(['2'], axis=1)
train2_y = train_data2['2']
val2_x = val_data2.drop(['2'], axis=1)
val2_y = val_data2['2']

train1_x = train_data1.drop(['1'], axis=1)
train1_y = train_data1['1']
val1_x = val_data1.drop(['1'], axis=1)
val1_y = val_data1['1']

In [4]:
import statsmodels.api as sm
from statsmodels.api import families

In [5]:
def get_accuracy_df(model, x, y, combo):
    pred_y = list((model.predict(x[combo])))

    accu_df = pd.DataFrame({'pred': pred_y, 'y': list(y)})

    accu_df['y_mean'] = np.mean(y)
    
    accu_df['TotErr'] = accu_df['y'] - accu_df['y_mean']
    accu_df['TotSqErr'] = np.power(accu_df['TotErr'], 2)

    accu_df['ResErr'] = accu_df['y'] - accu_df['pred']
    accu_df['ResSqErr'] = np.power(accu_df['ResErr'], 2)

    return accu_df

In [6]:
def get_bin_satur_llh(accu_df, link, n):

    llh = n*np.log(1)
    
    for row in accu_df.iterrows():
        p = link(row[1][1])

        llh += row[1][1]*np.log(p/(1-p)) + np.log(1-p)

    return llh

In [7]:
def get_bin_restrict_llh(accu_df, link, n):

    llh = n*np.log(1)
    
    for row in accu_df.iterrows():
        p = link(row[1][0])

        llh += row[1][1]*np.log(p/(1-p)) + np.log(1-p)


    return llh

In [8]:
def cloglog_link(y_hat):

    p = 1-np.exp(-np.exp(y_hat))

    return p

def logit_link(y_hat):
    
    p = 1/(1+np.exp(-y_hat))

    return p

In [9]:
model3_COLS = ['Behind Assists BTRS', 'Behind Assists OTRS', 'Contested Possessions BTRS', 'Contested Possessions OTRS', 'Disposals BTRS', 'Disposals OTRS', 'Effective Disposals BTRS', 'Effective Disposals OTRS', 'Handballs BTRS', 'Hitouts OTRS', 'Ineffective Disposals BTRS', 'Ineffective Disposals OTRS', 'Kicks BTRS', 'Kicks OTRS', 'Metres Gained BTRS', 'Metres Gained OTRS', 'Score Involvements BTRS', 'Score Involvements OTRS', 'Turnovers BTRS', 'Uncontested Possessions BTRS']

In [10]:
model3 = sm.GLM(train3_y, train3_x[model3_COLS], family = families.Binomial(link=families.family.Binomial.links[0]())).fit()

In [11]:
model2_COLS = ['Behind Assists BTRS', 'Contested Possessions BTRS', 'Contested Possessions OTRS', 'Disposals BTRS', 'Disposals OTRS', 'Effective Disposals BTRS', 'Effective Disposals OTRS', 'Hitouts OTRS', 'Ineffective Disposals BTRS', 'Kicks BTRS', 'Kicks OTRS', 'Metres Gained BTRS', 'Metres Gained OTRS', 'Score Involvements BTRS', 'Score Involvements OTRS', 'Turnovers OTRS', 'Uncontested Possessions BTRS']

In [12]:
model2 = sm.GLM(train2_y, train2_x[model2_COLS], family = families.Binomial(link=families.family.Binomial.links[0]())).fit()

In [13]:
model1_COLS = ['Behind Assists BTRS', 'Behind Assists OTRS', 'Bounces OTRS', 'Centre Clearances BTRS', 'Clangers BTRS', 'Clangers OTRS', 'Clearances BTRS', 'Clearances OTRS', 'Contested Marks OTRS', 'Contested Possessions BTRS', 'Contested Possessions OTRS', 'Disposals BTRS', 'Disposals OTRS', 'Effective Disposals BTRS', 'Effective Disposals OTRS', 'Frees Agains OTRS', 'Handballs BTRS', 'Handballs OTRS', 'Hitouts BTRS', 'Hitouts OTRS', 'Ineffective Disposals BTRS', 'Ineffective Disposals OTRS', 'Inside 50s BTRS', 'Inside 50s OTRS', 'Kicks BTRS', 'Kicks OTRS', 'Marks BTRS', 'Marks OTRS', 'Metres Gained BTRS', 'Metres Gained OTRS', 'One Percenters OTRS', 'Score Involvements BTRS', 'Score Involvements OTRS', 'Stoppage Clearances BTRS', 'Stoppage Clearances OTRS', 'Tackles BTRS', 'Turnovers BTRS', 'Turnovers OTRS', 'Uncontested Marks BTRS', 'Uncontested Possessions BTRS', 'Uncontested Possessions OTRS', 'Winloss']

In [14]:
model1 = sm.GLM(train1_y, train1_x[model1_COLS], family = families.Binomial(link=families.family.Binomial.links[0]())).fit()

In [15]:
import os
from collections import defaultdict as dd
manip_type = 'RankStandardisedData'

csv_list = os.listdir(f'../data/curated/{manip_type}')
csv_list.sort()

tally = dd(int)

data = pd.DataFrame()
for file in csv_list[1:]:
    if '2022' in file:
        data = pd.read_csv(f'../data/curated/{manip_type}/{file}')

        player = data['Player']
        pred3 = model3.predict(data[model3_COLS])
        pred2 = model2.predict(data[model2_COLS])
        pred1 = model1.predict(data[model1_COLS])
        pred = pd.DataFrame({'player': player, '3': pred3, '2': pred2, '1': pred1})

        three_votes = list(pred.sort_values('3', ascending = False)['player'])[0]

        two_votes = list(pred.sort_values('2', ascending = False)['player'])[0]
        if two_votes == three_votes:
            two_votes = list(pred.sort_values('2', ascending = False)['player'])[1]

        one_vote = list(pred.sort_values('1', ascending = False)['player'])[0]
        if one_vote in (three_votes, two_votes):
            one_vote = list(pred.sort_values('2', ascending = False)['player'])[1]
            
            if one_vote == two_votes:
                one_vote = list(pred.sort_values('2', ascending = False)['player'])[2]

        tally[three_votes] += 3
        tally[two_votes] += 2
        tally[one_vote] += 1
        

In [16]:
tally_list = list(tally.items())
tally_list.sort(key = lambda x:x[1], reverse=True)

In [17]:
tally_list

[('Brad Crouch', 22),
 ('Clayton Oliver', 21),
 ('Callum Mills', 20),
 ('Lachie Neale', 19),
 ('Touk Miller', 17),
 ('Christian Petracca', 16),
 ('Marcus Bontempelli', 16),
 ('Tom Lynch', 16),
 ('Luke Parker', 15),
 ('Jack Viney', 15),
 ('Patrick Cripps', 15),
 ('Oliver Wines', 15),
 ('Luke Ryan', 14),
 ('Jordan Dawson', 14),
 ('Thomas Liberatore', 14),
 ('Sam Walsh', 13),
 ('Steven May', 13),
 ('Brandan Parfitt', 12),
 ('Sam Docherty', 12),
 ('Dion Prestia', 12),
 ('Jackson Macrae', 12),
 ('Rory Laird', 12),
 ('Ben Keays', 12),
 ('Will Brodie', 12),
 ('Josh Dunkley', 12),
 ('Chad Warner', 11),
 ('Bailey Smith', 11),
 ('Patrick Dangerfield', 10),
 ('Darcy Parish', 10),
 ('Connor Rozee', 10),
 ('Thomas Stewart', 10),
 ('Travis Boak', 10),
 ('Zachary Merrett', 10),
 ('Cameron Guthrie', 9),
 ('Daniel Rich', 9),
 ('Hugh McCluggage', 9),
 ('Tim Kelly', 9),
 ('Jack Steele', 9),
 ('Tom Mitchell', 9),
 ('Isaac Heeney', 9),
 ('Charlie Curnow', 9),
 ('Andrew Brayshaw', 8),
 ('Matthew Kennedy', 8

In [18]:
import os
from collections import defaultdict as dd
manip_type = 'RankStandardisedData'

csv_list = os.listdir(f'../data/curated/{manip_type}')
csv_list.sort()

tally = dd(int)

data = pd.DataFrame()
for file in csv_list[1:]:
    if '2022' in file:
        data = pd.read_csv(f'../data/curated/{manip_type}/{file}')

        player = data['Player']
        pred3 = model3.predict(data[model3_COLS])
        pred2 = model2.predict(data[model2_COLS])
        pred1 = model1.predict(data[model1_COLS])
        pred = pd.DataFrame({'player': player, '3': pred3, '2': pred2, '1': pred1})

        three_votes = list(pred.sort_values('3', ascending = False)['player'])[0]

        two_votes = list(pred.sort_values('3', ascending = False)['player'])[1]

        one_vote = list(pred.sort_values('3', ascending = False)['player'])[2]

        tally[three_votes] += 3
        tally[two_votes] += 2
        tally[one_vote] += 1
        

In [19]:
tally_list = list(tally.items())
tally_list.sort(key = lambda x:x[1], reverse=True)

In [20]:
tally_list

[('Clayton Oliver', 30),
 ('Rory Laird', 22),
 ('Lachie Neale', 21),
 ('Brad Crouch', 20),
 ('Touk Miller', 20),
 ('Callum Mills', 20),
 ('Christian Petracca', 19),
 ('Tom Lynch', 19),
 ('Luke Parker', 18),
 ('Thomas Liberatore', 18),
 ('Marcus Bontempelli', 17),
 ('Patrick Cripps', 16),
 ('Josh Dunkley', 16),
 ('Sam Walsh', 15),
 ('Luke Ryan', 15),
 ('Will Brodie', 15),
 ('Jordan Dawson', 14),
 ('Ben Keays', 13),
 ('Cameron Guthrie', 13),
 ('Travis Boak', 13),
 ('Dion Prestia', 13),
 ('Chad Warner', 13),
 ('Oliver Wines', 13),
 ('Bailey Smith', 13),
 ('Brandan Parfitt', 12),
 ('Patrick Dangerfield', 12),
 ('Thomas Stewart', 12),
 ('Jy Simpkin', 12),
 ('Hugh McCluggage', 12),
 ('Jackson Macrae', 12),
 ('Noah Anderson', 12),
 ('Zachary Merrett', 12),
 ('Tom Mitchell', 12),
 ('Isaac Heeney', 12),
 ('Matt Rowell', 11),
 ('Jack Sinclair', 11),
 ('Sam Docherty', 11),
 ('Steven May', 11),
 ('Andrew Brayshaw', 10),
 ('Jordan De Goey', 10),
 ("Jaeger O'Meara", 10),
 ('Joel Selwood', 10),
 ('Ch